In [53]:
# Imports and Setup
from collections import defaultdict

import os
import mido

In [54]:
# Mapping of MIDI drum note numbers to Clone Hero drum notes and their corresponding flags
DRUM_MAPPING = {
    35: (0, 'K'),  # Acoustic Bass Drum mapped to note 0 with flag 'K'
    36: (0, 'K'),  # Bass Drum (Kick) mapped to note 0 with flag 'K'
    38: (1, 'R'),  # Acoustic Snare mapped to note 1 with flag 'R'
    40: (1, 'R'),  # Electric Snare mapped to note 1 with flag 'R'
    42: (2, 'Y'),  # Closed Hi-Hat mapped to note 2 with flag 'Y'
    44: (2, 'Y'),  # Pedal Hi-Hat mapped to note 2 with flag 'Y'
    46: (3, 'B'),  # Open Hi-Hat mapped to note 3 with flag 'B'
    49: (4, 'G'),  # Crash Cymbal 1 mapped to note 4 with flag 'G'
    51: (4, 'G'),  # Ride Cymbal mapped to note 4 with flag 'G'
    45: (2, 'Y'),  # Low Tom mapped to note 2 with flag 'Y'
    47: (2, 'Y'),  # Mid Tom mapped to note 2 with flag 'Y'
    48: (2, 'Y'),  # High Mid Tom mapped to note 2 with flag 'Y'
    50: (4, 'G'),  # High Tom mapped to note 4 with flag 'G'
    57: (4, 'G'),  # Crash Cymbal 2 mapped to note 4 with flag 'G'
}
# Example of expert cymbal mapping:
# 
# Below will output a yellow note at tick 95
# 95(tick) = N(note) 2(color)
#
# This will output a yellow cymbal at tick 95
# 95(tick) = N(note) 2(color)
# 95(tick) = N(note) 66(color)
# 
# 66 = yellow, 67 = blue, 68 = green

print(DRUM_MAPPING)


{35: (0, 'K'), 36: (0, 'K'), 38: (1, 'R'), 40: (1, 'R'), 42: (2, 'Y'), 44: (2, 'Y'), 46: (3, 'B'), 49: (4, 'G'), 51: (4, 'G'), 45: (2, 'Y'), 47: (2, 'Y'), 48: (2, 'Y'), 50: (4, 'G'), 57: (4, 'G')}


In [55]:
# File paths
midi_file_path = '../songs/midi_songs/the_spirit_of_radio_DRUMS.mid'
track_name = os.path.splitext(os.path.basename(midi_file_path))[0]
chart_path = f'../songs/chart_files/{track_name}.chart'

print(midi_file_path)
print(chart_path)
print(track_name)

../songs/midi_songs/the_spirit_of_radio_DRUMS.mid
../songs/chart_files/the_spirit_of_radio_DRUMS.chart
the_spirit_of_radio_DRUMS


In [56]:
# Create MIDI file object
if os.path.exists(midi_file_path):
    mid = mido.MidiFile(midi_file_path)
    print("Created Successfully")
else:
    print(f"File not found at {midi_file_path}")

Created Successfully


In [57]:
# Initialize song metadata
CHART_RESOLUTION = 192
song_metadata = {
    "Name": f"\"{track_name}\"",
    "Artist": "\"Unknown\"",
    "Charter": "\"ACE\"",
    "Album": "\"Generated Charts\"",
    "Year": "\"2024\"",
    "Offset": 0,
    "Resolution": CHART_RESOLUTION,
    "Player2": "\"bass\"",
    "Difficulty": 0,
    "PreviewStart": 0,
    "PreviewEnd": 0,
    "Genre": "\"Rock\""
}
print(song_metadata)

{'Name': '"the_spirit_of_radio_DRUMS"', 'Artist': '"Unknown"', 'Charter': '"ACE"', 'Album': '"Generated Charts"', 'Year': '"2024"', 'Offset': 0, 'Resolution': 192, 'Player2': '"bass"', 'Difficulty': 0, 'PreviewStart': 0, 'PreviewEnd': 0, 'Genre': '"Rock"'}


In [58]:
# Initialize data structure for chart
chart_data = {
    "Song": song_metadata,
    "SyncTrack": defaultdict(list),
    "Events": {},
    "ExpertDrums": defaultdict(list)
}

print(chart_data)

{'Song': {'Name': '"the_spirit_of_radio_DRUMS"', 'Artist': '"Unknown"', 'Charter': '"ACE"', 'Album': '"Generated Charts"', 'Year': '"2024"', 'Offset': 0, 'Resolution': 192, 'Player2': '"bass"', 'Difficulty': 0, 'PreviewStart': 0, 'PreviewEnd': 0, 'Genre': '"Rock"'}, 'SyncTrack': defaultdict(<class 'list'>, {}), 'Events': {}, 'ExpertDrums': defaultdict(<class 'list'>, {})}


In [59]:
merged_track = mido.merge_tracks(mid.tracks)

mid.length


297.7417085000018

In [60]:
# Append initial tempo and time signature to SyncTrack at 0
chart_data["SyncTrack"][0].append("TS 4")

# Now parse events in chronological order from the merged track
current_tick = 0

for msg in merged_track:
    current_tick += msg.time
    chart_tick = int(current_tick * CHART_RESOLUTION / mid.ticks_per_beat)

    if msg.type == 'set_tempo':
        midi_tempo = msg.tempo
        bpm = int(mido.tempo2bpm(midi_tempo))
        ch_tempo = bpm * 1000
        chart_data["SyncTrack"][chart_tick].append(f"B {ch_tempo}")
        print(ch_tempo)

    if msg.type == 'note_on' and msg.velocity > 0:
        if hasattr(msg, 'channel') and msg.channel == 9 and msg.note in DRUM_MAPPING:
            note_num, flag = DRUM_MAPPING[msg.note]
            note_str = f"N {note_num} 0{' ' + flag if flag else ''}"
            chart_data["ExpertDrums"][chart_tick].append(note_str)


134000
180000
144000
134000
240000
180000
76000
152000
76000
152000


In [61]:
# Build the .chart file text
chart_text = "[Song]\n{\n"
for key, value in chart_data["Song"].items():
    chart_text += f"  {key} = {value if isinstance(value, (int, float)) else f'{value}'}\n"
chart_text += "}\n"

chart_text += "[SyncTrack]\n{\n"
for tick in sorted(chart_data["SyncTrack"].keys()):
    for event in chart_data["SyncTrack"][tick]:
        chart_text += f"  {tick} = {event}\n"
        print(chart_text)
chart_text += "}\n"

chart_text += "[Events]\n{\n}\n"

chart_text += "[ExpertDrums]\n{\n"
for tick in sorted(chart_data["ExpertDrums"].keys()):
    for note in chart_data["ExpertDrums"][tick]:
        chart_text += f"  {tick} = {note[:-2]}\n"
chart_text += "}"

os.makedirs(os.path.dirname(chart_path), exist_ok=True)
with open(chart_path, 'w') as f:
    f.write(chart_text)

if os.path.exists(chart_path) and os.path.getsize(chart_path) > 0:
    print(f"Chart file successfully created at: {chart_path}")
    print("Chart file contains data")
else:
    print("Error: Failed to create chart file or file is empty")

[Song]
{
  Name = "the_spirit_of_radio_DRUMS"
  Artist = "Unknown"
  Charter = "ACE"
  Album = "Generated Charts"
  Year = "2024"
  Offset = 0
  Resolution = 192
  Player2 = "bass"
  Difficulty = 0
  PreviewStart = 0
  PreviewEnd = 0
  Genre = "Rock"
}
[SyncTrack]
{
  0 = TS 4

[Song]
{
  Name = "the_spirit_of_radio_DRUMS"
  Artist = "Unknown"
  Charter = "ACE"
  Album = "Generated Charts"
  Year = "2024"
  Offset = 0
  Resolution = 192
  Player2 = "bass"
  Difficulty = 0
  PreviewStart = 0
  PreviewEnd = 0
  Genre = "Rock"
}
[SyncTrack]
{
  0 = TS 4
  0 = B 134000

[Song]
{
  Name = "the_spirit_of_radio_DRUMS"
  Artist = "Unknown"
  Charter = "ACE"
  Album = "Generated Charts"
  Year = "2024"
  Offset = 0
  Resolution = 192
  Player2 = "bass"
  Difficulty = 0
  PreviewStart = 0
  PreviewEnd = 0
  Genre = "Rock"
}
[SyncTrack]
{
  0 = TS 4
  0 = B 134000
  6720 = B 180000

[Song]
{
  Name = "the_spirit_of_radio_DRUMS"
  Artist = "Unknown"
  Charter = "ACE"
  Album = "Generated Charts"
 